In [1]:
import json
import requests
import time
import re
import jieba
from collections import Counter

In [ ]:
url_templates = [
    "https://club.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98&productId=100007218425&score={}&sortType=5&page={}&pageSize=10&isShadowSku=0&fold=1"
]

In [ ]:
def get_comments(url):
    comments = []
    
    rsp = requests.get(url)
    rsp.encoding = "gbk"
    
    if rsp.status_code != 200:
        return []
    
    content = rsp.text
    if content:
        ind = content.find("(")
        s1 = content[ind+1:-2]
        #print(s1)
        try:
            js = json.loads(s1)
            comment_infos = js["comments"]
        except:
            print("Error")
            return ([])
        
        for comment_info in comment_infos:
            comment_content = comment_info["content"]
            str1 = comment_content + "\n"
            comments.append(str1)
            
    return comments

In [ ]:
good_comments = []
j = 0

for template in url_templates:
    for i in range(100):
        url = template.format(3, i)
        good_comments += get_comments(url)
        time.sleep(1)
        print("第{}条记录，文本总长度{}".format(j, len(good_comments)))
        j +=1

fw = open("good.txt", "w", encoding="utf-8")
fw.writelines(good_comments)

In [ ]:
bad_comments = []
j = 0

for template in url_templates:
    for i in range(100):
        url = template.format(1, i)
        bad_comments += get_comments(url)
        time.sleep(1)
        print("第{}条记录，文本总长度{}".format(j, len(bad_comments)))
        j +=1

fw = open("bad.txt", "w", encoding="utf-8")
fw.writelines(bad_comments)

In [2]:
def filter_punc(sentence):
    sentence = re.sub("[\s+\.\!\/_,$%^*(+\"\'“”《》?“]+|[+——！，。？、~@#￥%……&*（）：]+", "", sentence)  
    return(sentence)

In [3]:
def prepare_data(good_file, bad_file, is_filter = True):
    all_words = []
    pos_sentences = []
    neg_sentences = []
    with open(good_file, "r", encoding="utf-8") as fr:
        for idx, line in enumerate(fr):
            if is_filter:
                line = filter_punc(line)
            words = jieba.lcut(line)
            if len(words) > 0:
                all_words += words
                pos_sentences.append(words)
    print("{0} 包含 {1} 行, {2} 个词.".format(good_file, idx+1, len(all_words)))

    count = len(all_words)
    with open(bad_file, "r", encoding="utf-8") as fr:
        for idx, line in enumerate(fr):
            if is_filter:
                line = filter_punc(line)
            words = jieba.lcut(line)
            if len(words) > 0:
                all_words += words
                neg_sentences.append(words)
    print("{0} 包含 {1} 行, {2} 个词.".format(bad_file, idx+1, len(all_words)-count))

    dic = {}
    cnt = Counter(all_words)
    for word, freq in cnt.items():
        dic[word] = [len(dic), freq]
    print('字典大小：{}'.format(len(dic)))
    return(pos_sentences, neg_sentences, dic)

In [4]:
def word2index(word, diction):
    if word in diction:
        value = diction[word][0]
    else:
        value = -1
    return(value)

In [5]:
def index2word(index, diction):
    for w,v in diction.items():
        if v[0] == index:
            return(w)
    return(None)

In [6]:
good_file = "good.txt"
bad_file  = "bad.txt"

pos_sentences, neg_sentences, diction = prepare_data(good_file, bad_file, True)
st = sorted([(v[1], w) for w, v in diction.items()])
#print(st)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Erik\AppData\Local\Temp\jieba.cache
Loading model cost 0.917 seconds.
Prefix dict has been built succesfully.


good.txt 包含 3309 行, 61497 个词.
bad.txt 包含 1354 行, 18912 个词.
字典大小：7213


In [7]:
def sentence2vec(sentence, dictionary):
    vector = np.zeros(len(dictionary))
    for l in sentence:
        vector[l] += 1
    return(1.0 * vector / len(sentence))